# Interactive Storyteller


## Connect to Furhat

In [15]:
from pathlib import Path
import json
from furhat_remote_api import FurhatRemoteAPI
furhat = FurhatRemoteAPI("localhost")

In [18]:
furhat.set_face(character='Titan', mask="Adult")
furhat.set_voice(name='Joanna')

{'message': 'Successfully changed Furhat voice', 'success': True}

In [ ]:
DATA_PATH = Path("../data/processed/stories")

## Load stories

In [ ]:
def load_all_stories():
    stories = {}
    for path in DATA_PATH.glob("*.json"):
        story_id = path.stem
        with open(path, "r") as f:
            stories[story_id] = json.load(f)
    return stories

STORIES = load_all_stories()
list(STORIES.keys())


['stardust_arena', 'abandoned_station', 'lantern_keeper']

## Load emotions

In [21]:
def get_all_emotions(stories):
    emos = set()
    for story in stories.values():
        for scene in story["scenes"]:
            emos.update(scene.get("templates", {}).keys())
    return sorted(emos)

EMOTIONS = get_all_emotions(STORIES)
EMOTIONS


['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprised']

## Helpers

In [22]:
def get_scene(story, scene_id):
    for scene in story["scenes"]:
        if scene["id"] == scene_id:
            return scene
    return None

def choose_template(scene, emotion):
    """Pick template for emotion, fall back to neutral or description."""
    return scene["templates"].get(
        emotion,
        scene["templates"].get("neutral", scene["description"])
    )

def next_scene(scene, chosen_option_id):
    for opt in scene.get("options", []):
        if opt["id"] == chosen_option_id:
            return opt["nextScene"]
    return None


## Generic story selection

In [ ]:
def select_story(mood: str, emotion: str, stories=STORIES):
    """
    Choose the story whose targetMoods best match mood/emotion.
    """
    best_id = None
    best_score = -1

    for sid, story in stories.items():
        target = set(story.get("targetMoods", []))
        score = 0
        if mood in target:
            score += 2
        if emotion in target:
            score += 1
        if score > best_score:
            best_score = score
            best_id = sid

    if best_id is None:
        best_id = next(iter(stories.keys()))
    return best_id


## Mood and intent parsing from Furhat speech

In [ ]:
KNOWN_MOODS = [
    "tired", "comfort-seeking", "sad",
    "excited", "happy", "energized",
    "neutral", "curious"
] # fix to match emotions?

def detect_mood_from_text(text: str) -> str:
    t = text.lower()
    if "comfort" in t:
        return "comfort-seeking"
    for m in KNOWN_MOODS:
        if m in t:
            return m
    # fallback
    return "neutral"


def detect_emotion_from_text(text: str) -> str:
    """
    Placeholder until the webcam model is integrated.
    Maps some common words to your emotion labels.
    """
    t = text.lower()
    if any(w in t for w in ["happy", "great", "good", "nice"]):
        return "happy"
    if any(w in t for w in ["sad", "down", "bad"]):
        return "sad"
    if any(w in t for w in ["angry", "mad", "annoyed"]):
        return "angry"
    if any(w in t for w in ["scared", "afraid", "fear", "nervous"]):
        return "fear"
    if any(w in t for w in ["disgust", "gross"]):
        return "disgust"
    if any(w in t for w in ["surprised", "wow"]):
        return "surprised"
    return "neutral"


## Listen helper

In [26]:
def listen_text(language: str = "en-US") -> str:
    """Listen once and return lowercase text, or '' if nothing."""
    response = furhat.listen(language=language)
    if response and getattr(response, "message", None):
        return response.message.strip()
    return ""


## Pick option from spoken answer

In [27]:
def choose_option_from_speech(scene, text: str):
    """
    Try to map user's utterance to an option:
    - number words (one, two, three...)
    - digits (1, 2, 3)
    - keywords from the option text
    Returns option object or None.
    """
    t = text.lower()

    options = scene.get("options", [])
    if not options:
        return None

    # digit index
    for i, opt in enumerate(options, start=1):
        if str(i) in t:
            return opt

    # word -> index
    words_to_num = {
        "one": 1, "first": 1,
        "two": 2, "second": 2,
        "three": 3, "third": 3
    }
    for w, num in words_to_num.items():
        if w in t and 1 <= num <= len(options):
            return options[num - 1]

    # keyword match: if option text words appear
    for opt in options:
        key = opt["text"].split()[0].lower()  # very simple
        if key in t:
            return opt

    # fallback: first option
    return options[0]


## Full microphone driven story session

In [28]:
def run_story_session():
    # --- Greeting & mood ---
    furhat.say(text="Hello! I am your interactive storyteller.", blocking=True)
    furhat.say(text="How are you feeling right now?", blocking=True)
    mood_text = listen_text()
    mood = detect_mood_from_text(mood_text)

    # --- TEMP: ask for emotion verbally (until webcam model is ready) ---
    furhat.say(text="If I looked at your face, would you say you feel happy, sad, angry, afraid, disgusted, surprised, or neutral?", blocking=True)
    emo_text = listen_text()
    emotion = detect_emotion_from_text(emo_text)
    if emotion not in EMOTIONS:
        emotion = "neutral"

    # --- Select story ---
    story_id = select_story(mood, emotion, STORIES)
    story = STORIES[story_id]
    furhat.say(text=f"I will tell you a story called {story['name']}.", blocking=True)

    # intro = first scene in JSON
    scene_id = story["scenes"][0]["id"]

    while True:
        scene = get_scene(story, scene_id)
        if scene is None:
            furhat.say(text="Oops, I lost the story. Let's stop here for now.", blocking=True)
            break

        # read scene with emotional template
        text_to_say = choose_template(scene, emotion)
        furhat.say(text=text_to_say, blocking=True)

        # no options -> end
        if not scene.get("options"):
            furhat.say(text="That was the end of this story.", blocking=True)
            break

        # read options
        furhat.say(text="What would you like to do next?", blocking=True)
        for idx, opt in enumerate(scene["options"], start=1):
            furhat.say(text=f"Option {idx}: {opt['text']}", blocking=True)

        # listen for choice / meta-intents
        answer = listen_text()

        # global intents
        if "quit" in answer or "stop" in answer:
            furhat.say(text="Okay, I will stop the story here.", blocking=True)
            break

        if "change" in answer or "another" in answer:
            # re-select story but keep same mood/emotion
            new_story_id = select_story(mood, emotion, STORIES)
            if new_story_id != story_id:
                story_id = new_story_id
                story = STORIES[story_id]
                scene_id = story["scenes"][0]["id"]
                furhat.say(text=f"Let's try another story: {story['name']}.", blocking=True)
                continue
            else:
                furhat.say(text="We will stay with the same story for now.", blocking=True)

        if "repeat" in answer:
            # just repeat current scene
            furhat.say(text="Let me repeat that part.", blocking=True)
            continue

        # choose option
        opt = choose_option_from_speech(scene, answer)
        scene_id = opt["nextScene"]


In [29]:
run_story_session()